In [1]:
import numpy as np
import pandas as pd
import math
from numpy import delete
from tqdm import tqdm
import random

In [2]:
users = pd.read_csv('u.user', sep='|', names=['u_id','age','sex', 'occupation','zip_code'], encoding='latin-1')
ratings = pd.read_csv(r'u.data',sep='\t', names=['u_id','m_id','rating','unix_timestamp'],encoding='latin-1')
movies = pd.read_csv('u.item',sep='|',names=['m_id','title','release_date','video_release_date','imdb_url'],usecols=range(5),encoding='latin-1')
movie_ratings = pd.merge(movies, ratings) 
alls = pd.merge(movie_ratings, users)
print(alls)

       m_id                                   title release_date  \
0         1                        Toy Story (1995)  01-Jan-1995   
1         4                       Get Shorty (1995)  01-Jan-1995   
2         5                          Copycat (1995)  01-Jan-1995   
3         7                   Twelve Monkeys (1995)  01-Jan-1995   
4         8                             Babe (1995)  01-Jan-1995   
5         9                 Dead Man Walking (1995)  01-Jan-1995   
6        11                    Seven (Se7en) (1995)  01-Jan-1995   
7        12              Usual Suspects, The (1995)  14-Aug-1995   
8        15               Mr. Holland's Opus (1995)  29-Jan-1996   
9        17              From Dusk Till Dawn (1996)  05-Feb-1996   
10       19                   Antonia's Line (1995)  01-Jan-1995   
11       21           Muppet Treasure Island (1996)  16-Feb-1996   
12       22                       Braveheart (1995)  16-Feb-1996   
13       23                      Taxi Driver (19

[100000 rows x 12 columns]


In [3]:
print('用户数:{0},---电影的数量:{1}<<<<'.format(len(set(alls['u_id'])),len(set(alls['m_id']))))

用户数:943,---电影的数量:1682<<<<


In [4]:
print('设置一个矩阵,横向是用户ID，纵向是电影的ID')
user_movie = np.zeros([943,1682])
for i in range(len(ratings.values)):
    user_movie[ratings.values[i][0]-1][ratings.values[i][1]-1]=ratings.values[i][2]
    
print('user_movie:',user_movie[:2,:])

设置一个矩阵,横向是用户ID，纵向是电影的ID
user_movie: [[ 5.  3.  4. ...,  0.  0.  0.]
 [ 4.  0.  0. ...,  0.  0.  0.]]


In [60]:
print('''
传入用户数据表data,测试用户ID，
''')
def user_nice(data, user_id, K):
    user = data[user_id]
    delete(data,user_id,axis=0)
    user_nn = np.zeros([len(data),len(data)])
    near_u = {}
    for i in tqdm(range(len(data))):
        near_u.update({cos_(user,data[i]):i})
    nice_ls = sorted(zip(near_u.keys(),near_u.values()))
    nice_out = []
    for j in range(80):   #len(nice_ls[:100])
        count = 0
        # 这里得到 最优的用户id
        this_one = data[nice_ls[j][1]]
        p = np.transpose(np.nonzero(this_one))
        l = np.transpose(np.nonzero(user))
        for n in p:
            if n not in l:
                nice_out+=[movies.values[n]]
                count+=1
            if count == K:
                print('数量足够一共{0},分别是{1}'.format(K,nice_out))
                return nice_out
#     print('没有足够数量，最后得到{0}'.format(nice_out))
#     return nice_out


传入用户数据表data,测试用户ID，



In [48]:
print('''
这个是余弦相似性公式： x，y均为行向量表示，返回x与y的相似度
''')
def cos_(x, y):
    x_ = np.ndarray([len(user_movie[0])])
    for i in range(len(user_movie[0])):
        x_[i] = x[i]
    y_ = np.ndarray([len(user_movie[0])])
    for i in range(len(user_movie[0])):
        y_[i] = y[i]
    
    return (np.sum(x_.T*y))/(np.sqrt(np.sum(x.T*x)*np.sum(y.T*y)))
# print(cos_(user_movie[1],user_movie[110]))


这个是余弦相似性公式： x，y均为行向量表示，返回x与y的相似度



In [61]:
dss=list(user_nice(user_movie, 100, 3))

100%|██████████████████████████████████████████████████████████████████████████████| 943/943 [00:00<00:00, 1093.83it/s]


数量足够一共3,分别是[array([[14, 'Postino, Il (1994)', '01-Jan-1994', nan,
        'http://us.imdb.com/M/title-exact?Postino,%20Il%20(1994)']], dtype=object), array([[15, "Mr. Holland's Opus (1995)", '29-Jan-1996', nan,
        "http://us.imdb.com/M/title-exact?Mr.%20Holland's%20Opus%20(1995)"]], dtype=object), array([[28, 'Apollo 13 (1995)', '01-Jan-1995', nan,
        'http://us.imdb.com/M/title-exact?Apollo%2013%20(1995)']], dtype=object)]


In [94]:
dss

[array([[14, 'Postino, Il (1994)', '01-Jan-1994', nan,
         'http://us.imdb.com/M/title-exact?Postino,%20Il%20(1994)']], dtype=object),
 array([[15, "Mr. Holland's Opus (1995)", '29-Jan-1996', nan,
         "http://us.imdb.com/M/title-exact?Mr.%20Holland's%20Opus%20(1995)"]], dtype=object),
 array([[28, 'Apollo 13 (1995)', '01-Jan-1995', nan,
         'http://us.imdb.com/M/title-exact?Apollo%2013%20(1995)']], dtype=object)]

In [83]:
train = pd.read_csv('u1.base', sep='\t', names=['u_id','m_id','rating', 'unix_timestamp'], encoding='latin-1')
# test = pd.read_csv(r'u1.test',sep='\t', names=['u_id','m_id','rating','unix_timestamp'], encoding='latin-1')
train_list = np.zeros([943, 1682])
for i in range(len(train.values)):
    train_list[train.values[i][0]-1][train.values[i][1]-1] = train.values[i][2]
test_list = np.zeros([943, 1682])
for i in range(len(test.values)):
    test_list[test.values[i][0]-1][test.values[i][1]-1] = test.values[i][2]
    
print(train_list[100][:20])
print(test_list[100][:20])

[ 3.  0.  0.  0.  0.  0.  3.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.]


In [113]:
right = 0
error = 0
for i in range(2):
    u__ = random.randint(10,900)
    result = list(user_nice(train_list, u__, 5))
    for j in result:
        if j[0][0] in np.transpose(np.nonzero(train_list[u__])):
            right+=1
        else:
            error+=1

100%|██████████████████████████████████████████████████████████████████████████████| 943/943 [00:00<00:00, 1092.56it/s]


数量足够一共5,分别是[array([[300, 'Air Force One (1997)', '01-Jan-1997', nan,
        'http://us.imdb.com/M/title-exact?Air+Force+One+(1997)']], dtype=object), array([[315, 'Apt Pupil (1998)', '23-Oct-1998', nan,
        'http://us.imdb.com/Title?Apt+Pupil+(1998)']], dtype=object), array([[316, 'As Good As It Gets (1997)', '23-Dec-1997', nan,
        'http://us.imdb.com/Title?As+Good+As+It+Gets+(1997)']], dtype=object), array([[331, 'Edge, The (1997)', '26-Sep-1997', nan,
        'http://us.imdb.com/M/title-exact?Edge%2C+The+(1997/I)']], dtype=object), array([[339, 'Mad City (1997)', '01-Jan-1997', nan,
        'http://us.imdb.com/M/title-exact?Mad+City+(1997)']], dtype=object)]


100%|██████████████████████████████████████████████████████████████████████████████| 943/943 [00:00<00:00, 1107.98it/s]


数量足够一共5,分别是[array([[288, 'Scream (1996)', '20-Dec-1996', nan,
        'http://us.imdb.com/M/title-exact?Scream%20(1996)']], dtype=object), array([[292, 'Rosewood (1997)', '21-Feb-1997', nan,
        'http://us.imdb.com/M/title-exact?Rosewood%20(1997)']], dtype=object), array([[294, 'Liar Liar (1997)', '21-Mar-1997', nan,
        'http://us.imdb.com/Title?Liar+Liar+(1997)']], dtype=object), array([[324, 'Lost Highway (1997)', '21-Feb-1997', nan,
        'http://us.imdb.com/Title?Lost+Highway+(1997)']], dtype=object), array([[898, 'Postman, The (1997)', '01-Jan-1997', nan,
        'http://us.imdb.com/M/title-exact?imdb-title-119925']], dtype=object)]


In [115]:
print('正确率为{0}'.format(right/(right+error)))

正确率为0.2


In [ ]:
print('召回率{0}'.format())

In [87]:
random.randint(10,20)

14